In [ ]:
# Import pyspark functions
from pyspark.sql.functions import *
# Import URL processing
import urllib

In [ ]:
# Check tables in filestore to get name of credentials file
dbutils.fs.ls("/FileStore/tables")

In [ ]:
## Read the csv file containing the AWS keys to Databricks 
# specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to Spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Extract the access key and secret access key from the spark dataframe created. The secret access key will be encoded using urllib.parse.quote for security purposes. safe="" means that every character will be encoded."""
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secret key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
# Mount the S3 bucket by passing in the S3 URL and the desired mount name to dbutils.fs.mount().
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e35b2767ae1-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e35b2767ae1-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [ ]:
# To check if the S3 bucket was mounted succesfully run the following command:
display(dbutils.fs.ls("/mnt/user-0e35b2767ae1-bucket/topics"))

In [ ]:
topics =  {'df_pin':'0e35b2767ae1.pin', 'df_geo':'0e35b2767ae1.geo', 'df_user':'0e35b2767ae1.user'}
for df, topic in topics.items():
    file_location = f"/mnt/user-0e35b2767ae1-bucket/topics/{topic}/partition=0/*.json" 
    file_type = "json"
    # Spark infers the schema
    infer_schema = "true"
    # Read in JSONs from mounted S3 bucket
    df_name = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    # Display Spark dataframe to check its content
    display(df_pin)
    display(df_geo)
    display(df_user)

In [ ]:
## Unmount S3 bucket
# If you want to unmount the S3 bucket, run the following code:

dbutils.fs.unmount("/mnt/mount_name")

## Cleaning the dataframes and sorting columns


In [ ]:
# Cleaning the dataframes and sorting columns

df_pin = df_pin.dropDuplicates()
df_pin = df_pin.withColumnRenamed('index', 'ind') 
# Create function to convert null or bad values to None
def convert_to_null(df, column, bad_values):
    '''Converts no or bad values in dataframe columns to null '''
    df = df.withColumn(column, when(col(column).like(bad_values), None).otherwise(col(column)))
    return df
# Create a dictionary of columns and values to be replaced, then replace them with None in the df
bad_values_dict = {
    "description": "No description available%",
    "follower_count": "User Info Error",
    "image_src": "Image src error.",
    "poster_name": "User Info Error",
    "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
    "title": "No Title Data Available"
}
# Call the function while iterating through the dictionary items.
for key, value in bad_values_dict.items():
    df_pin = convert_to_null(df_pin, key, value)
# Convert non numeric data in the follower_count column to numbers
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
# Cast all columns with numbers only to integer type
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast('int'))
df_pin = df_pin.withColumn('downloaded', df_pin['downloaded'].cast('int'))
# Convert save_location column to include only the save location path
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))
# Re-order the dataframe columns.
df_pin = df_pin.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category", "downloaded")
df_pin.na.drop(how = "all")

## Clean df_geo data

In [ ]:
# Remove duplicates
df_geo = df_geo.dropDuplicates()
# Form an array column from latitude and longitude columns and drop them from the DataFrame
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))
df_geo = df_geo.drop('latitude', 'longitude')
# Cast column timestamp to Timestamp type
df_geo = df_geo.withColumn("timestamp", df_geo["timestamp"].cast('Timestamp'))
# Re-order the dataframe columns.
df_geo = df_geo.select("ind", "country", "coordinates", "timestamp")

## Clean df_user data

In [ ]:
df_user = df_user.dropDuplicates()
df_user = df_user.withColumn("user_name", concat_ws(' ', ("first_name"), ("last_name")))
df_user = df_user.drop("first_name", "last_name")
df_user = df_user.withColumn('date_joined', df_user['date_joined'].cast('Timestamp'))
df_user = df_user.select("ind", "user_name", "age", "date_joined")

# Querying the data

Find the most popular Pinterest category people post to based on their country.

In [ ]:
# Join the pin and geo df on ind, group by columns, count all rows. 
df_pin.join(df_geo, 'ind').groupBy('country', 'category')\
 .agg(count('*').alias('count'))\
 .groupBy('country')\
 .agg(max(struct('count', 'category')).alias('max_count')) \
 .select('country','max_count.category','max_count.count' ).show()

In [ ]:
# Cell shows categories according to their popularity in all country

In [ ]:
from pyspark.sql.functions import *
df_pin.join(df_geo, 'ind').groupBy('country', 'category')\
    .agg(count('*').alias('category_count')) \
        .groupBy('country', 'category')\
        .agg(max('category_count').alias('max_category_count')) \
        .select ('country', 'category','max_category_count')\
        .sort(desc("max_category_count")).show()\



 Find how many posts each category had between 2018 and 2022

In [ ]:
from pyspark.sql.functions import *

# Join df_pin with df_geo on 'ind' where year is within given timestamp range
df_category_by_specific_years = df_pin.join(df_geo, on='ind') \
    .where((year('timestamp') >= 2018) & (year('timestamp') <= 2022))\
    .groupBy(year('timestamp').alias('post_year'), 'category') \
    .agg(count('*').alias('category_count'))\
    .show()

In [ ]:
# Join df_pin with df_geo on 'ind' where year is within given timestamp range
df_category_by_specific_years = df_pin.join(df_geo, on='ind')\
    .where((year('timestamp') >= 2018) & (year('timestamp') <= 2022))\
    .groupBy(year('timestamp').alias('post_year'), 'category')\
    .agg(count('*').alias('post_count'))\
    .groupBy('category') \
    .agg(sum('post_count'))\
    .show()

For each country find the user with the most followers.

In [ ]:
# Find the country with the user that has the most follower
# Step 1
df_most_followers_by_country = df_pin.join(df_geo, 'ind')\
 .groupBy('country')\
 .agg(max(struct('follower_count', 'poster_name')).alias('max'))\
 .selectExpr('country', 'max.poster_name', 'max.follower_count')\
 .orderBy(col('follower_count').desc())
df_most_followers_by_country.show()
# Step 2
df_pin.join(df_geo, 'ind').groupBy('country').agg(max(struct('follower_count', 'poster_name')).alias('max')) \
 .selectExpr('country', 'max.poster_name', 'max.follower_count').orderBy(col('follower_count').desc()).limit(1).show()

7. What is the most popular category people post to based on the following age groups:

In [ ]:
from pyspark.sql.functions import *
df_max_popular_cat_posted_to_by_age_group = df_pin.join(df_user, 'ind').withColumn('age_group', expr("""case
 WHEN age BETWEEN 18 AND 24 THEN '18-24'
 WHEN age BETWEEN 25 AND 35 THEN '25-35'
 WHEN age BETWEEN 36 AND 50 THEN '36-50'
 WHEN age> 50 THEN '+50' END """))\
.groupBy('age_group', 'category') \
.agg(count('*').alias('count')) \
.groupBy('age_group') \
.agg(max(struct('count', 'category')).alias('max_count')) \
.select('age_group', 'max_count.category', 'max_count.count') \
.show()

8. The median follower count for users in the following age groups:

In [ ]:
from pyspark.sql.functions import *
df_median_followercount_by_age_group = df_pin.join(df_user, 'ind')\
    .withColumn('age_group', expr("""case\
 WHEN age BETWEEN 18 AND 24 THEN '18-24'
 WHEN age BETWEEN 25 AND 35 THEN '25-35'
 WHEN age BETWEEN 36 AND 50 THEN '36-50'
 WHEN age> 50 THEN '+50' END """))\
    .groupBy("age_group") \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .select("age_group", "median_follower_count").show()

9. Find how many users have joined between 2015 and 2020.

In [ ]:
from pyspark.sql.functions import *
# Join df_pin with df_geo on 'ind' where year is within given timestamp range
df_users_joined_by_years = df_geo.join(df_user, on='ind') \
    .where((year('timestamp') >= 2015) & (year('timestamp') <= 2020))\
    .groupBy(year('timestamp').alias('post_year'))\
    .agg(count('*').alias('number_users_joined'))\
    .show()


Find the median follower count of users have joined between 2015 and 2020.

post_year, a new column that contains only the year from the timestamp column
median_follower_count, a new column containing the desired query output

In [ ]:
df_median_followercount_by_years = df_user.join(df_pin, 'ind')\
    .groupBy(year('date_joined').alias('post_year'))\
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .select('post_year', 'median_follower_count')\
    .where(col('post_year').between('2015', '2020')).show()

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.

In [ ]:
df_user.join(df_pin, 'ind')\
    .withColumn('age_group', expr("""case\
 WHEN age BETWEEN 18 AND 24 THEN '18-24'
 WHEN age BETWEEN 25 AND 35 THEN '25-35'
 WHEN age BETWEEN 36 AND 50 THEN '36-50'
 WHEN age> 50 THEN '+50' END """))\
    .groupBy(year('date_joined').alias('post_year'), 'age_group')\
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .select('post_year','age_group', 'median_follower_count')\
    .where(col('post_year').between('2015', '2020')).orderBy('post_year','age_group').show()